# 第七章 函数装饰器和闭包

## 7.1 装饰器基础知识

In [1]:
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


In [2]:
target

<function __main__.deco.<locals>.inner()>

## 7.2 Python何时执行装饰器

In [3]:
registry = []

def register(func):
    print("running register(%s)" %func)
    registry.append(func)
    return func

@register
def f1():
    print("running f1()")

@register
def f2():
    print("running f2()")

def f3():
    print("running f3()")

running register(<function f1 at 0x000001A59C3EA660>)
running register(<function f2 at 0x000001A59C3EAAC0>)


In [4]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

> 装饰器在被装饰器定义之后立即执行

In [5]:
_  = f1(), f2(), f3()

running f1()
running f2()
running f3()


* 装饰器函数通常在一个模块中定义, 然后应用到其他的模块中的函数上
* 多数装饰器会在内部定义一个函数, 然后将其返回

## 7.3 使用装饰器改进策略模式

## 7.4 变量作用域

In [6]:
def f1(a):
    print(a, b)

In [7]:
f1(3)

NameError: name 'b' is not defined

In [ ]:
b = 3
f1(3)

3 3


> b 没有定义时报错

In [ ]:
b = 6
def f2(a):
    print(f"running f2({a})", end= " ")
    print(a, b)
    b = 9
f2(3)

running f2(3) 

UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [ ]:
def f2_(a):
    print(f"running f2_({a})", end= " ")
    print(a, b)
f2_(3)

running f2_(3) 3 6


> 当b作为局部变量, 由于在函数内给它赋了值, 所以Python 会尝试从本地环境获取b。事实是, 函数在运行前首先会编译其定义体

In [ ]:
b = 6
def f3(a):
    global b
    print(a, b)
    b = 10
f3(4)
b

4 6


10

> 使用 global 变量, 可以让解释器把b当作全局变量

In [ ]:
from dis import dis
dis(f1)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_FAST                0 (a)
             16 LOAD_GLOBAL              2 (b)
             28 PRECALL                  2
             32 CALL                     2
             42 POP_TOP
             44 LOAD_CONST               0 (None)
             46 RETURN_VALUE


In [ ]:
dis(f2)

  2           0 RESUME                   0

  3           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_CONST               1 ('running f2(')
             16 LOAD_FAST                0 (a)
             18 FORMAT_VALUE             0
             20 LOAD_CONST               2 (')')
             22 BUILD_STRING             3
             24 LOAD_CONST               3 (' ')
             26 KW_NAMES                 4
             28 PRECALL                  2
             32 CALL                     2
             42 POP_TOP

  4          44 LOAD_GLOBAL              1 (NULL + print)
             56 LOAD_FAST                0 (a)
             58 LOAD_FAST                1 (b)
             60 PRECALL                  2
             64 CALL                     2
             74 POP_TOP

  5          76 LOAD_CONST               5 (9)
             78 STORE_FAST               1 (b)
             80 LOAD_CONST               0 (None)
             82 RETURN_VALUE


## 7.5 闭包

> 闭包指延伸了作用域的函数, 其中包括函数体中引用、但是不在定义体中定义的非全局变量。 函数是不是匿名没关系, 关键是它可以访问定义体之外定义的非全局变量

示例 7-8 计算移动平均值的类

In [ ]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [ ]:
avg = Averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

示例 7-9 计算移动平均值的高阶函数

In [ ]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [ ]:
avg = make_averager()
avg(-10), avg(10), avg(9) 

(-10.0, 0.0, 3.0)

示例 7-11 审查make_averager

In [ ]:
avg.__code__.co_varnames, avg.__code__.co_freevars, avg.__code__.co_cellvars

(('new_value', 'total'), ('series',), ())

## 7.6 nonlocal 声明

示例 7-13 计算移动平均值的高阶函数, 不保存所有历史值, 但有缺陷

In [ ]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [ ]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

> count += 1 时, 会把 count 变成局部变量, 

示例 7-14 计算移动平均值, 不保存所有历史 (使用nonlocal修正)

In [ ]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [ ]:
avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

## 7.7 实现一个简单的装饰器

示例 7-15 一个简单的装饰器, 输出函数的运行时间

In [ ]:
import time
def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


示例 7-16 使用clock装饰器

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [ ]:
snooze(0.0002)
factorial(6)

[0.00053668s] snooze(0.0002) -> None
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720


720

In [ ]:
factorial.__name__, snooze.__name__

('clocked', 'clocked')

示例 7-17 改进后的clock装饰器

In [ ]:
from functools import wraps

def clock(func):
    @wraps(func)
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [ ]:
factorial.__name__, snooze.__name__

('factorial', 'snooze')

## 7.8 标准库中的装饰器

### 7.8.1 使用functools.lru_cache 做备忘

In [ ]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [ ]:
fibonacci(6)

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00000000s] fibonacci(6) -> 8


8

In [ ]:
fibonacci(4)

3

In [ ]:
@functools.lru_cache(maxsize=1, typed=False)
@clock
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [ ]:
fibonacci(4)

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3


3

### 7.8.2 单分派泛函数

In [8]:
import html
from ipywidgets import HTML

def htmlize(obj):
    content = html.escape(repr(obj))
    return "<pre>{}</pre>".format(content)

HTML(htmlize({1, 2, 3}))

HTML(value='<pre>{1, 2, 3}</pre>')

In [9]:
HTML(htmlize(["alpha", 66, {3, 2, 1}]))

HTML(value='<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>')

In [10]:
htmlize(["alpha", 66, {3, 2, 1}])

'<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>'

示例 7-21 singledispatch 创建一个自定义的htmlize.register 装饰器, 把多个函数绑在一起组成一个泛函数

In [11]:
from functools import singledispatch
from collections import abc
import numbers
import html


@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)


@htmlize.register(str)  # <2>
def _(text):  # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)


@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)


@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [12]:
HTML(htmlize(("Hello", "World",)))

HTML(value='<ul>\n<li><p>Hello</p></li>\n<li><p>World</p></li>\n</ul>')

## 7.9 叠放装饰器

In [14]:
def d1(func):
    def inner(*argc):
        print("d1")
        return func(*argc)
    return inner
def d2(func):
    def inner(*argc):
        print("d2")
        return func(*argc)
    return inner
@d2
@d1
def f():
    print('f')

In [15]:
f()

d2
d1
f


In [16]:
def f():
    print('f')
f = d2(d1(f))
f()

d2
d1
f


## 7.10 参数化装饰器

示例 7-22 示例 7-2 中 registration.py 模块的删减版，这里再次给出是为了便于讲解

In [17]:
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x000001A59CB6B740>)
running main()
registry -> [<function f1 at 0x000001A59CB6B740>]
running f1()


### 7.10.1 一个参数化的注册装饰器

示例 7-23 为了接受参数, 新的register装饰器必须作为函数调用

In [19]:
registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x000001A59CB6B6A0>)
running register(active=True)->decorate(<function f2 at 0x000001A59CB6BCE0>)


## 7.11 本章小结

<img src="./images/第七章总结.jpg" width="70%">